In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import collections

#fields=['SubjectID','ProblemID','FirstCorrect','time','norm','avg_prior_percent_correct','avg_prior_percent_correct_prob','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','18','19','22']
fields_sub=['FirstCorrect','avg_prior_percent_correct','1','2','3','4','5','6', '7','8', '9', '10', '11', '12', '13', '14','15', '16', '18','19','22']
df_data=pd.read_csv('data_challenge.csv', sep='\t')

df_data.replace('-', np.nan, inplace=True)
probs=['1', '2', '3', '4', '5', '6', '7','8', '9', '10', '11', '12', '13', '14','15', '16', '18','19','22']
df_data[probs] = df_data[probs].apply(pd.to_numeric, errors='coerce', axis=1)
df_data.fillna(df_data.mean(),inplace=True)
print (df_data.head(5))

   Unnamed: 0 SubjectID  ProblemID  FirstCorrect  EverCorrect  time    norm  \
0           0      S002         10          True         True   0.0  0.0000   
1           1      S002         16          True         True   0.0  0.0000   
2           2      S002         18         False         True  16.0  0.8500   
3           3      S002         22         False         True   5.0  0.7222   
4           4      S002         13         False         True   0.0  0.0000   

   avg_prior_percent_correct  avg_prior_percent_correct_prob        1  \
0                    0.63282                          15.152  0.23624   
1                    1.00000                          75.000  0.23624   
2                    1.00000                          40.909  0.23624   
3                    0.66700                          36.364  0.23624   
4                    0.50000                          38.710  0.23624   

     ...           10        11        12        13        14        15  \
0    ...   

In [14]:
d = collections.OrderedDict()
for p in probs:
    if p in ['15','18','22']:
        continue
    print (p)
    
    df=df_data.loc[(df_data['ProblemID'] == int(p))]
    df=df[fields_sub]
    df.drop(columns=[p],inplace=True)
    df_train, df_test = train_test_split(df, test_size=0.2)
    df_train_y=df_train[['FirstCorrect']]
    df_train.drop(columns=['FirstCorrect'],inplace=True)
    df_test_y=df_test[['FirstCorrect']]
    df_test.drop(columns=['FirstCorrect'],inplace=True)
    pval=[2,2]
    flag=0
    temp=0
    
    while max(pval)>0.1:
        if len(pval)==1:
            flag=1
            break
        X2 = sm.add_constant(df_train)
        est = sm.OLS(df_train_y.astype(float), X2.astype(float))
        est2 = est.fit()
        pval=round(est2.pvalues,4)
    
        if pval.index[0]=='const':
            pval.drop(labels='const',inplace=True)
        if len(pval)==3:
            temp=pval.index
        idx=pval.idxmax()
        
        pval.drop(labels=idx,inplace=True)
        df_train.drop(labels=idx,inplace=True,axis=1)   
    
     
    if flag==1:
        d[p]=temp
    else:
        d[p]=pval.index
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
16
19


In [15]:
print (d)

OrderedDict([('1', Index(['13', '14', '16'], dtype='object')), ('2', Index(['1', '4', '5', '8', '9', '10', '11', '12', '13', '14', '15', '19',
       '22'],
      dtype='object')), ('3', Index(['4', '5', '7', '8', '9', '10', '11', '12', '13', '14', '15', '19',
       '22'],
      dtype='object')), ('4', Index(['avg_prior_percent_correct', '1'], dtype='object')), ('5', Index(['6'], dtype='object')), ('6', Index(['5', '8', '9', '10', '12', '15', '22'], dtype='object')), ('7', Index(['5', '9', '10', '12', '15', '19', '22'], dtype='object')), ('8', Index(['4', '5', '7', '13', '14'], dtype='object')), ('9', Index(['4', '5'], dtype='object')), ('10', Index(['4', '7', '9', '13', '22'], dtype='object')), ('11', Index(['5', '6', '7', '8', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '22'],
      dtype='object')), ('12', Index(['avg_prior_percent_correct', '2', '5', '8', '10', '11', '19', '22'], dtype='object')), ('13', Index(['1', '2', '3', '6', '8', '10', '11', '14', '15', '16',